In [ ]:
# Run this to import all needed files and libraries
%pip install ultralytics
!git clone https://github.com/LuckierBread/YOLOv9-Car-Counter

%cd /content/YOLOv9-Car-Counter

import cv2
import pandas as pd
import numpy as np
from ultralytics import YOLO
from tracker import *
from google.colab.patches import cv2_imshow
model = YOLO('yolov9c.pt')

class_list = ['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']

In [2]:
# Function to calculate the distence between a set of points and a given line
def optimized_point_to_segment_distance(line_start, line_end, points):
  #convert to numpy arrays for efficency
  line_start = np.array(line_start) #[1,1]
  line_end = np.array(line_end) #[1,1]
  points = np.array(points) #[:,2]

  # Vector from start to end of the line segment
  v = line_end - line_start
  # Calc v_norm for the denominator of projection
  v_norm = np.dot(v, v)

  # Vectors from start of the line segment to each point
  u = points - line_start

  # Calc projection scalar lengths
  proj_len = np.dot(u, v) / v_norm
  # Calc where the projection actually falls on the line segment
  # >0 means it is after the line start and <=1 means it is before the end.
  # on_segment will act as a mask for the next calculations
  on_segment = (proj_len >= 0) & (proj_len <= 1)

  # Calculate projected points only for those on the line segment
  # Calc closest point on the line
  proj_points = line_start + np.outer(proj_len[on_segment], v)
  # Calc the normal distance from the line to the point
  dist_to_proj = np.linalg.norm(proj_points - points[on_segment], axis=1)

  # Distance to endpoints for those outside the segment
  # Calc distance from start
  dist_to_start = np.linalg.norm(points[~on_segment] - line_start, axis=1)
  # Calc distance from end
  dist_to_end = np.linalg.norm(points[~on_segment] - line_end, axis=1)

  # Combine distances into a single array
  distances = np.empty_like(proj_len)
  distances[on_segment] = dist_to_proj
  distances[~on_segment] = np.minimum(dist_to_start, dist_to_end)

  return distances

In [ ]:
tracker=Tracker()
frame_count=0
#grab frames from video
cap=cv2.VideoCapture('trafficclip.mp4')
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))

size = (frame_width, frame_height)
output = cv2.VideoWriter('output.avi',  cv2.VideoWriter_fourcc(*'MJPG'), 10, size)
#prep id lists
down={}
up={}
#prep counters
counter_down=set()
counter_up=set()

#predefine colors just because
white_color = (255,255,255)  # white color for text
red_color = (0, 0, 255)  # (B, G, R)
blue_color = (255,0,0)  #(B, G, R)
black_color = (0,0,0)
yellow_color = (0,255,255)

while True:
    ret,frame = cap.read()
    if not ret:
        break
    frame_count += 1

    results=model.predict(frame)

    a=results[0].boxes.data
    a = a.detach().cpu().numpy()
    px=pd.DataFrame(a).astype("float")
    #print(px)

    vehicle_list=[]

    for index,row in px.iterrows():
        #print(row)
        x1=int(row[0])
        y1=int(row[1])
        x2=int(row[2])
        y2=int(row[3])
        d=int(row[5])
        c=class_list[d]
        if 'car' or 'motorcycle' in c:
            vehicle_list.append([x1,y1,x2,y2])


    bbox_id=tracker.update(vehicle_list)
    #print(bbox_id)
    for bbox in bbox_id:
      x3,y3,x4,y4,id=bbox
      cx=int(x3+x4)//2
      cy=int(y3+y4)//2
      cv2.circle(frame,(cx,cy),4,red_color,-1) #draw ceter points of bounding box
      cv2.rectangle(frame, (x3, y3), (x4, y4), black_color, 2)  # Draw bounding box
      cv2.putText(frame,str(id),(cx,cy),cv2.FONT_HERSHEY_COMPLEX,0.8,yellow_color,2) #lable item

      #Counter Conditions
      up_line = [600,250,1000,250]
      down_line = [100,350,450,350]
      offset = 10

      #Up Condition
      if optimized_point_to_segment_distance(up_line[0:2], up_line[2:4],[cx,cy])<10:
        up[id]=[cx,cy]

      #Down Condition
      if optimized_point_to_segment_distance(down_line[0:2], down_line[2:4],[cx,cy])<10:
        down[id]=[cx,cy]

      if id in up:
        cv2.circle(frame,(cx,cy),4,red_color,-1)
        cv2.rectangle(frame, (x3, y3), (x4, y4), red_color, 2)
        cv2.putText(frame,str(id),(cx,cy),cv2.FONT_HERSHEY_COMPLEX,0.8,yellow_color,2)
        counter_up.add(id)

      if id in down:
        cv2.circle(frame,(cx,cy),4,blue_color,-1)
        cv2.rectangle(frame, (x3, y3), (x4, y4), blue_color, 2)
        cv2.putText(frame,str(id),(cx,cy),cv2.FONT_HERSHEY_COMPLEX,0.8,yellow_color,2)
        counter_down.add(id)



    # Draw Lines
    # print(up)
    cv2.line(frame,up_line[0:2],up_line[2:4],red_color,3)  #  starting cordinates and end of line cordinates
    cv2.putText(frame,('Up Line'),up_line[0:2],cv2.FONT_HERSHEY_SIMPLEX, 0.5, yellow_color, 1, cv2.LINE_AA)

     # print(down)
    cv2.line(frame,down_line[0:2],down_line[2:4],blue_color,3)  #  starting cordinates and end of line cordinates
    cv2.putText(frame,('Down Line'),down_line[0:2],cv2.FONT_HERSHEY_SIMPLEX, 0.5, yellow_color, 1, cv2.LINE_AA)

    upwards = (len(counter_up))
    downwards = (len(counter_down))

    cv2.putText(frame,('going up - ')+ str(upwards),(60,20),cv2.FONT_HERSHEY_SIMPLEX, 1, red_color, 2, cv2.LINE_AA)
    cv2.putText(frame,('going down - ')+ str(downwards),(60,60),cv2.FONT_HERSHEY_SIMPLEX, 1, red_color, 2, cv2.LINE_AA)

    #cv2_imshow(frame)
    output.write(frame)
    if cv2.waitKey(1)&0xFF==27:
        break
output.release()
cap.release()
cv2.destroyAllWindows()


In [ ]:
_